In [1]:
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.losses import SparseCategoricalCrossentropy
import tensorflow as tf
import numpy as np
import glob
import matplotlib.pyplot as plt
import glob
import cv2

In [2]:
d_shape = 256
def pre_processing(d_shape = d_shape):

    path = '/Users/luizeduardosantos/Documents/Turing/projeto-trainee-cv/'


    images = {}
    labels = {}
    for part in ["train", "validation", "test"]:
        Darth_images = np.array([cv2.imread(file) for file in glob.glob(path + part + "/Darth Vader" + '/*')])
        Storm_images = np.array([cv2.imread(file) for file in glob.glob(path + part + "/Stormtrooper" + '/*')])
        Yoda_images = np.array([cv2.imread(file) for file in glob.glob(path + part + "/Yoda" + '/*')])

        labels_Darth = np.full((Darth_images.shape[0], 1), 0)
        labels_Storm = np.full((Storm_images.shape[0], 1), 1)
        labels_Yoda = np.full((Yoda_images.shape[0], 1), 2)

        labels[part] = np.concatenate((labels_Darth,labels_Storm,labels_Yoda))

        part_images = np.concatenate((Darth_images, Storm_images, Yoda_images))

        resized_images = np.empty((part_images.shape[0],d_shape,d_shape,3), dtype=int)

        for i in range (part_images.shape[0]):
            resized_images[i] = cv2.resize(part_images[i],(d_shape,d_shape))

        resized_images = resized_images/255.

        images[part] = resized_images

    return images, labels


images, labels = pre_processing()

    

/Users/luizeduardosantos/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  # Remove the CWD from sys.path while we load stuff.
/Users/luizeduardosantos/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  # This is added back by InteractiveShellApp.init_path()
/Users/luizeduardosantos/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (wh

In [4]:
model2 = Sequential()
model2.add(layers.Conv2D(32, (12,12), activation='relu', input_shape=(256, 256 ,3)))
model2.add(layers.MaxPooling2D((2,2)))

model2.add(layers.Conv2D(16, (24,24), activation='relu'))
model2.add(layers.MaxPooling2D((2,2)))

model2.add(layers.Conv2D(8, (48,48), activation='relu'))
model2.add(layers.Flatten())

model2.add(layers.Dense(32, activation= 'relu'))
model2.add(layers.Dense(8, activation= 'relu'))
model2.add(layers.Dense(3, activation= 'softmax'))
model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 245, 245, 32)      13856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 122, 122, 32)      0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 99, 99, 16)        294928    
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 49, 49, 16)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 2, 2, 8)           294920    
_________________________________________________________________
flatten_1 (Flatten)          (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)               

In [5]:

model2.compile(optimizer='adam',
              loss= SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

history = model2.fit(x = images["train"],y = labels["train"], epochs=9, batch_size = 64,
                    validation_data = (images["validation"], labels["validation"]), shuffle = True )

                    
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')



Epoch 1/9
23/23 [==============================] - 397s 17s/step - loss: 1.5655 - accuracy: 0.3456 - val_loss: 1.0684 - val_accuracy: 0.3770
Epoch 2/9
23/23 [==============================] - 404s 18s/step - loss: 1.0537 - accuracy: 0.4030 - val_loss: 1.0759 - val_accuracy: 0.3607
Epoch 3/9
23/23 [==============================] - 400s 17s/step - loss: 1.0359 - accuracy: 0.4139 - val_loss: 0.9802 - val_accuracy: 0.4699
Epoch 4/9
23/23 [==============================] - 461s 20s/step - loss: 0.9562 - accuracy: 0.5123 - val_loss: 0.9165 - val_accuracy: 0.5628
Epoch 5/9
11/23 [=============>................] - ETA: 3:25 - loss: 0.9082 - accuracy: 0.6009

In [ ]:
model2.evaluate(images["test"])

NameError: name 'model2' is not defined

In [ ]:
model2.save('/Users/luizeduardosantos/Documents/Turing/Interpretando-CNNs/model2-files/')

INFO:tensorflow:Assets written to: /Users/luizeduardosantos/Documents/Turing/Interpretando-CNNs/model2-files/assets
